# IEOR4004 Project 2 - Optimized NBA Schedule

## Imports

Please put all imports up here for consistency

In [ ]:
import pandas as pd

## Q1. 

In [ ]:
games_df = pd.read_csv('./data/games.csv')
games_df.head()

In [ ]:
teams = pd.unique(games_df[["Visitor", "Home"]].values.ravel())
print(teams)

In [ ]:
summary_dict = {}
for team in teams:
    team_summary = {}
    
    # Home dates 
    home_dates = games_df.loc[games_df["Home"] == team, "Date"].tolist()
    team_summary['Home dates'] = home_dates
    
    # Away dates
    away_dates = games_df.loc[games_df["Visitor"] == team, "Date"].tolist()
    team_summary['Away dates'] = away_dates
    
    # Home counts vs all other teams
    home_counts = (
        games_df.loc[games_df["Home"] == team, "Visitor"]
        .value_counts()
        .rename_axis("Opponent")
        .reset_index(name="Home Games")
    )
    # Away counts vs all other teams
    away_counts = (
        games_df.loc[games_df["Visitor"] == team, "Home"]
        .value_counts()
        .rename_axis("Opponent")
        .reset_index(name="Away Games")
    )
    total_counts = pd.merge(home_counts,
                            away_counts,
                            on="Opponent",
                            how="outer").fillna(0)
    team_summary['H/A Games vs. Others'] = total_counts

    summary_dict[team] = team_summary
    print(f"\n {team} Summary:")
    print(f"Home dates: {team_summary['Home dates']}")
    print(f"Away dates: {team_summary['Away dates']}")
    print("Head-to-head counts:")
    print(team_summary['H/A Games vs. Others'])

## Q2.

### Integer Programming Formulation for NBA Schedule Feasibility

#### Sets

- $T$: set of teams  
- $D$: set of dates  

#### Parameters

- $H_{i,d} \in \{0,1\}$:  equals 1 if team $i$ originally played **at home** on date $d$

- $A_{i,d} \in \{0,1\}$:  equals 1 if team $i$ originally played **away** on date $d$

- $h_{i,j}$:  number of times team $i$ originally played team $j$ **at home**

- $a_{i,j}$:  number of times team $i$ originally played team $j$ **away**  

#### Decision Variables

- $z_{d,i,j} \in \{0,1\}$:  
  equals 1 if on date $d$, team $i$ is the **home team** and plays against team $j$


#### Constraints

##### Home dates must stay the same (1)
$$
\sum_{\substack{j\in T\\ j \ne i}} z_{d,i,j} \;=\; H_{i,d}\qquad\forall d\in D,\ \forall i\in T,\.
$$

##### Away dates must stay the same (2)
$$
\sum_{\substack{j\in T\\ j \ne i}} z_{d,j,i} \;=\; A_{i,d}\qquad\forall d\in D,\ \forall i\in T,\.
$$

##### No two teams can both be home against each other on the same date (3)
$$
z_{d,i,j} + z_{d,j,i} \;\le\; 1\qquad\forall d\in D,\ j\ne i.
$$

##### Season home-counts vs each opponent must match the original schedule (4)
$$
\sum_{d\in D} z_{d,i,j} \;=\; h_{i,j}\qquad\forall i\in T,\ \forall j\in T,\ j\ne i..
$$

##### Season away-counts vs each opponent must match the original schedule (5)
$$
\sum_{d\in D} z_{d,j,i} \;=\; a_{i,j}\qquad\forall i\in T,\ \forall j\in T,\ j\ne i.
$$

##### Integer/Binary Constraint (6)
$$
z_{d,i,j} \in \{0,1\}\qquad\forall d\in D,\ \forall i\in T,\ \forall j\in T,\ j\ne i.
$$